<h2> Retrieve a list of URLs (downloading paths)</h2>
<li> http://www.neeq.com.cn/disclosure/announcement.html, filter it by keyword “理财”, and start & end date，to get the exact number of pages on required reports (e.g., '17' pages for dates between 2017.11.15-2017.12.31 ).
<li>run the code below，see a small pop-up window “请输入下载页数:", type that page number in the blank, then click 'return/enter'.

In [1]:
##url
import requests, json, os
from urllib.request import urlretrieve
urllist=list()


if __name__ == '__main__':
    url = 'http://www.neeq.com.cn/disclosureInfoController/infoResult.do?callback=jQuery18309654062013135463_1505374211021'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36',
        'Referer': 'http://www.neeq.com.cn/disclosure/announcement.html',
        'Origin': 'http://www.neeq.com.cn'}
    page_num = input('请输入下载的页数:')
    for num in range(int(page_num)):
        data = {'disclosureType':'5',
            'page':'%d' % num,
            'isNewThree':'1',
            'startTime':'2017-11-15',
            'endTime':'2017-12-31',
            'keyword':'理财'}

        req = requests.post(url=url, headers=headers, data=data)
        html = json.loads(req.text[41:][:-1])
        for each in html[0]['listInfo']['content']:           
            urllist.append("http://www.neeq.com.cn" + each['destFilePath'])


请输入下载的页数:1


In [2]:
len(urllist)##338

20

<h2>Download pdfs

In [ ]:
for i in range(338):
    url = urllist[i]
    fullfilename = os.path.join('/Users/binsair/Desktop/FT/理财公告2', str(i+2) +'.pdf')#match its row number in csv/excel file with a header
    urlretrieve(url, fullfilename)
    

<h2>Web scrap announcement date and company name

In [ ]:
##date
import requests, json, os
from urllib.request import urlretrieve
date=list()


if __name__ == '__main__':
    url = 'http://www.neeq.com.cn/disclosureInfoController/infoResult.do?callback=jQuery18309654062013135463_1505374211021'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36',
        'Referer': 'http://www.neeq.com.cn/disclosure/announcement.html',
        'Origin': 'http://www.neeq.com.cn'}
    page_num = input('请输入下载的页数:')
    for num in range(int(page_num)):
        data = {'disclosureType':'5',
            'page':'%d' % num,
            'isNewThree':'1',
            'startTime':'2017-11-15',
            'endTime':'2017-12-31',
            'keyword':'理财'}

        req = requests.post(url=url, headers=headers, data=data)
        html = json.loads(req.text[41:][:-1])
        for each in html[0]['listInfo']['content']:
            date.append(each['publishDate'])
            

In [ ]:
##firm
import requests, json, os
from urllib.request import urlretrieve
firm=list()


if __name__ == '__main__':
    url = 'http://www.neeq.com.cn/disclosureInfoController/infoResult.do?callback=jQuery18309654062013135463_1505374211021'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36',
        'Referer': 'http://www.neeq.com.cn/disclosure/announcement.html',
        'Origin': 'http://www.neeq.com.cn'}
    page_num = input('请输入下载的页数:')
    for num in range(int(page_num)):
        data = {'disclosureType':'5',
            'page':'%d' % num,
            'isNewThree':'1',
            'startTime':'2017-11-15',
            'endTime':'2017-12-31',
            'keyword':'理财'}

        req = requests.post(url=url, headers=headers, data=data)
        html = json.loads(req.text[41:][:-1])
        for each in html[0]['listInfo']['content']:
            firm.append(each['companyName'])
            

<h2>Write 'date', 'firm' and 'urls' into a csv file

In [ ]:
rawlist=list()
for i in range(len(urllist)):
    rawlist.append((date[i], firm[i], urllist[i]))

In [ ]:
import csv

outfile = open("raw.csv", "w", encoding='utf_8_sig')

outfile.write("Announcement Date, Firm, url\n")

for r in rawlist:
    outfile.write("%s, %s, %s\n" % r)
outfile.close()

<h2>Convert pdf, returns its text content as a string

In [ ]:
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt, nltk

In [ ]:
def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 

In [ ]:
import nltk
from nltk import sent_tokenize,word_tokenize 
#sentences = nltk.Text(sent_tokenize(text))
text = convert('/Users/binsair/Desktop/FT/理财公告2/'+str(2)+'.pdf', pages= None)
striptext = text.replace('\n\n', ' ')
striptext = striptext.replace('\n', ' ')
#words = word_tokenize(striptext)
sentences = sent_tokenize(striptext)
#words
#sentences
striptext

<h2>Get city info

In [ ]:
##NR: not readable by pyPDF; NA: can be read, but in an unfriendly format; NC cid doc format
cities = list()
def corp(words):
    if words[1].find('cid') == 0:
        city = 'NC'
    else:
        for j in range(10):
            if words[j] == '证':
                city = 'NA'
        
            elif words[j][-6::] == "股份有限公司" or words[j][-3::] == "子公司":
                if words[j-1] == ')':
                    city = words[j-2]
                
                elif words[j-1] != ')' and '（' in words[j]:
                    city = words[j][words[j].index('（')+1: words[j].index('（')+ 3]
                elif words[j-1] != ')' and '（' not in words[j]:
                    city = words[j][0:2]
                else:
                    city = 'NA'
    return city
           



In [ ]:
##city

##NR: not readable by pyPDF; NA: can be read, but in an unfriendly format; NC cid doc format

from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
import pprint
from nltk import word_tokenize

cities = list()
for i in range(338):
    try:
        text = convert('/Users/binsair/Desktop/FT/理财公告2/'+str(i+2)+'.pdf', pages= None)
        striptext = text.replace('\n\n', ' ')
        striptext = striptext.replace('\n', ' ')
        words = word_tokenize(striptext)
        city = corp(words)
        cities.append(city)

         
    except:
        cities.append('NR')

    



len(cities)

<h2>Separate city and province (dictionary</h2>
<li>use an extra 'cityfull.xlsx' file to map each to a city or province.
<li>info in 'cityfull.xlsx' might be obsolete (‘县’/'市'). Manually revise small part.

In [ ]:
#!pip3 install xlrd
import csv

#citydic = dict()
#with open('/Users/binsair/Desktop/FT/cityfull.csv') as csvDataFile:
#    csvReader = csv.reader(csvDataFile)
#    for row in csvReader:
#        print(row)
#dt = pd.read_excel('/Users/binsair/Desktop/FT/cityfull.xlsx').to_dict()

import collections as co
import pandas as pd

df = pd.read_excel('/Users/binsair/Desktop/FT/cityfull.xlsx')
df = df.where(pd.notnull(df), None)
od = co.OrderedDict((k.strip(),v.strip()) 
                    for (k,v) in df.values)

In [ ]:
for m in range(len(cities)):
    if cities[m] in od:
        full = od[cities[m]]
        cities[m] = full       

In [ ]:
import csv

outfile = open("city.csv", "w", encoding='utf_8_sig')

#outfile.write("Expected return\n")

for r in cities:
    outfile.write("%s\n" % r)
outfile.close()

<h2>Amount of Purchase

In [ ]:
def stoin(a):
   # a = list(a)
    a = [x for x in a if x != ',']

    magic = lambda nums: int(''.join(str(i) for i in nums))
    a = magic(a)

    return a

In [ ]:
def stoin2(a):
    if a[-3:] == '.00':##元 单位
        a = a[0:-3]
        if len(a) >= 5: 
            return int(str(stoin(a))[0:-4])
        else:
            return 1
    else:
        return stoin(a)
    

In [ ]:
def com(a, b):
    if '亿' in a and '亿' not in b:
        c = a[:-2]
        d = b[:-2]
        if stoin2(c)*10000 >= stoin2(d):
            return a
        else:
            return b
    elif '亿' not in a and '亿' in b:
        c = a[:-2]
        d = b[:-2]
        if stoin2(c) >= stoin2(d)*10000:
            return a
        else:
            return b
    else:
        c = a[:-2]
        d = b[:-2]
        if stoin2(c) >= stoin2(d):
            return a
        else:
            return b        
        



In [ ]:
com('1000,00000000.00元的','500万元')
com('1亿的', '1000,00000000.00元的')
com('1亿的', '500万元')

In [ ]:


def purchase (words):
    y = ''
    b = list()
    if words[1].find('cid') == 0:
        y = 'NC'
    else:
            b = list()
            for j in range(len(words)-2):     
                if words[j+2][1:2] == '元' and words[j][-2::] == '超过' \
                or words[j+2][1:2] == '元' and words[j][-1] == "币" \
                or words[j+2][1:2] == '元' and words[j][-4::] == "过人民币"\
                or words[j+2][1:2] == '元' and words[j][-6::] == "额度为人民币"\
                or words[j+2][1:2] == '元' and words[j-1][-4::] == '投资额度'\
                or words[j+2][1:2] == '元' and words[j-2][-4::] == '投资额度'\
                or words[j+2][0:1] == '元' and words[j][-2::] == '超过' \
                or words[j+2][0:1] == '元' and words[j][-1] == "币" \
                or words[j+2][0:1] == '元' and words[j][-4::] == "过人民币"\
                or words[j+2][0:1] == '元' and words[j][-6::] == "额度为人民币"\
                or words[j+2][0:1] == '元' and words[j-1][-4::] == '投资额度'\
                or words[j+2][0:1] == '元' and words[j-2][-4::] == '投资额度'\
                or words[j+2][1:2] == '元' and words[j] == '过' and words[j-1][-2::] == '不超'\
                or words[j+2][0:1] == '元' and words[j] == '过' and words[j-1][-2::] == '不超'\
                or words[j+2][2:3] == '元' and words[j][-3::] == '不超过'\
                or words[j+2][2:3] == '元' and words[j][-1] == "币"\
                or words[j+2][2:3] == '元' and words[j-1][-4::] == '投资额度'\
                or words[j+2][2:3] == '元' and words[j-2][-4::] == '投资额度'\
                or words[j+2][2:3] == '元' and words[j] == '过' and words[j-1][-2::] == '不超'\
                or words[j+2][1:2] == '元' and words[j][-2::] == '资金'\
                or words[j+2][0:1] == '元' and words[j][-2::] == '资金'\
                or words[j+2][2:3] == '元' and words[j][-2::] == '资金':
                    x = words[j+1] + words[j+2][0:2]
                    b.append(x)
                    if len(b) ==1:
                        y = b[0]
                    elif len(b) == 2:
                        y = com(b[0], b[1])
                        b = [y]
                        
                        
                        
                elif '元' in words[j+2] and '过' in words[j+2]:
                        if '币' in words[j+2]:
                            x = words[j+2][words[j+2].index('币')+1: words[j+2].index('元')+1]
                            
                            b.append(x)
                            if len(b) ==1:
                                y = b[0]
                            elif len(b) == 2:
                                y = com(b[0], b[1])
                                b = [y]
                        else:
                            x = words[j+2][words[j+2].index('过')+1: words[j+2].index('元')+1]
                            b.append(x)
                            if len(b) ==1:
                                y = b[0]
                            elif len(b) == 2:
                                y = com(b[0], b[1])
                                b = [y] 
                            
                            
                        #else:
                         #   y = words[j+2][words[j+2].index('过')+1: words[j+2].index('元')+1]
    
  
    
    
    

    
    
    
    
                elif '元' in words[j+2] and '过' in words[j+2] and any(word in words[j+2] for word in ['总额', '金额', '额度']):
                    if '币' in words[j+2]:
                        x = words[j+2][words[j+2].index('币')+1: words[j+2].index('元')+1]
                        
                            
                        b.append(x)
                        if len(b) ==1:
                            y = b[0]
                        elif len(b) == 2:
                            y = com(b[0], b[1])
                            b = [y]
                    else:
                        x = words[j+2][words[j+2].index('过')+1: words[j+2].index('元')+1]
                        b.append(x)
                        if len(b) ==1:
                            y = b[0]
                        elif len(b) == 2:
                            y = com(b[0], b[1])
                            b = [y] 
                        
                        
                    #else:
                        #y = words[j+2][words[j+2].index('过')+1: words[j+2].index('元')+1]
    
    
                elif '元' in words[j+2] and '过' in words[j+2] and any(word in words[j+3] for word in ['总额', '金额', '额度']):
                    if '币' in words[j+2]:
                        x = words[j+2][words[j+2].index('币')+1: words[j+2].index('元')+1]
                        
                        b.append(x)
                        if len(b) ==1:
                            y = b[0]
                        elif len(b) == 2:
                            y = com(b[0], b[1])
                            b = [y]
                    else:
                        x = words[j+2][words[j+2].index('过')+1: words[j+2].index('元')+1]
                        b.append(x)
                        if len(b) ==1:
                            y = b[0]
                        elif len(b) == 2:
                            y = com(b[0], b[1])
                            b = [y] 
                        
                        
                        
                    #else:
                        #y = words[j+2][words[j+2].index('过')+1: words[j+2].index('元')+1]
                

                
                
                

                
                
                elif words[j+2][0] == '元' and '额' in words[j+3] and '超' in words[j-1]\
                or words[j+2][0] == '元' and '额' in words[j+2] and '超' in words[j-1]\
                or words[j+2][0] == '元' and '额' in words[j+4] and '超' in words[j-1]:
                    if '币' in words[j]:
                        x = words[j][words[j].index('币')+1:]+ words[j+1] + '元'
                        
                        b.append(x)
                        if len(b) ==1:
                            y = b[0]
                        elif len(b) == 2:
                            y = com(b[0], b[1])
                            b = [y]
                    else:
                        x = words[j]+ words[j+1] + '元'
                        b.append(x)
                        if len(b) ==1:
                            y = b[0]
                        elif len(b) == 2:
                            y = com(b[0], b[1])
                            b = [y]
                
                
                elif words[j+2][1:2] == '元' and '额' in words[j+1] and '：' in words[j+1]:
                    x = words[j+1][words[j+1].index('：')+1:]+ words[j+2][0:2]
                    b.append(x)
                    if len(b) ==1:
                        y = b[0]
                    elif len(b) == 2:
                        y = com(b[0], b[1])
                        b = [y]                    
                         
                elif '元' in words[j+2] and '额' in words[j+2] and '：' in words[j+2]:
                    if '币' in words[j+2]:
                        x = words[j+2][words[j+2].index('币')+1: words[j+2].index('元')+1]
                        
                        b.append(x)
                        if len(b) ==1:
                            y = b[0]
                        elif len(b) == 2:
                            y = com(b[0], b[1])
                            b = [y]
                    else:
                        x = words[j+2][words[j+2].index('：')+1: words[j+2].index('元')+1]
                        b.append(x)
                        if len(b) ==1:
                            y = b[0]
                        elif len(b) == 2:
                            y = com(b[0], b[1])
                            b = [y]
                            
                            
                elif '元' in words[j+2] and '额' in words[j+2] and '超' in words[j+2]:
                    if '币' in words[j+2]:
                        x = words[j+2][words[j+2].index('币')+1: words[j+2].index('元')+1]
                        
                        b.append(x)
                        if len(b) ==1:
                            y = b[0]
                        elif len(b) == 2:
                            y = com(b[0], b[1])
                            b = [y]
                    else:
                        x = words[j+2][words[j+2].index('过')+1: words[j+2].index('元')+1]
                        b.append(x)
                        if len(b) ==1:
                            y = b[0]
                        elif len(b) == 2:
                            y = com(b[0], b[1])
                            b = [y]
                  #  else:
                    #    y = words[j]+ words[j+1] + '元'
            
        
    return y

In [ ]:
##amount purchase

##NR: not readable by pyPDF; NA: can be read, but in an unfriendly format; NC cid doc format

from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
import pprint
from nltk import word_tokenize

qlist = list()
for i in range(338):
    try:
        text = convert('/Users/binsair/Desktop/FT/理财公告2/'+str(i+2)+'.pdf', pages= None)
        striptext = text.replace('\n\n', ' ')
        striptext = striptext.replace('\n', ' ')
        words = word_tokenize(striptext)
        b = purchase(words)
        qlist.append(b)
             
                
    except:
        qlist.append('NR')

    



len(qlist)

In [ ]:
len([i for i,x in enumerate(qlist) if x == ''])
len([i for i,x in enumerate(qlist) if x == 'NR'])

In [ ]:
import csv

outfile = open("rawpurchase.csv", "w", encoding='utf_8_sig')

outfile.write("Amount of purchase\n")

for r in qlist:
    outfile.write("%s\n" % r)
outfile.close()

<h2>Estimated rate of return

In [ ]:
def percent (words):
    y = ''
    if words[1].find('cid') == 0:
        y = 'NC'
    else:
            b = list()
            for j in range(len(words)): 
                if '%' in words[j]:
                    h = words[j-1] + words[j] +', '
                    y += h
                    
    return y

In [ ]:
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
import pprint
from nltk import word_tokenize

flist = list()
for i in range(338):
    try:
        text = convert('/Users/binsair/Desktop/FT/理财公告2/'+str(i+2)+'.pdf', pages= None)
        striptext = text.replace('\n\n', ' ')
        striptext = striptext.replace('\n', ' ')
        words = word_tokenize(striptext)
        b = percent(words)
        flist.append(b)
             
                
    except:
        flist.append('NR')

    



len(flist)

In [ ]:
import csv

outfile = open("rawpercent.csv", "w", encoding='utf_8_sig')

outfile.write("Expected return\n")

for r in flist:
    outfile.write("%s\n" % r)
outfile.close()

<h2> Product category

In [ ]:
def category (words):
    p = list()
    for j in range(len(words)):     
        if '产品类型' in words[j]:
            p.append(words[j][words[j].index('型')+2:])
        elif '产品类型' not in words[j] and '类' in words[j] and not p:
            p.append(words[j])
        elif '品种' in words[j]:
            p.append(words[j+1])
        else:
            pass
    return p

In [ ]:
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
import pprint
from nltk import word_tokenize

glist = list()
for i in range(338):
#for i in range(140,160):
        try:
            text = convert('/Users/binsair/Desktop/FT/理财公告2/'+str(i+2)+'.pdf', pages= None)
            striptext = text.replace('\n\n', ' ')
            striptext = striptext.replace('\n', ' ')
            words = word_tokenize(striptext)
            b = category(words)
            glist.append(b)
             
                
        except:
            glist.append(['python not readable'])

    



len(glist)

In [ ]:
import pandas as pd
df = pd.DataFrame(glist)
#df
df.to_csv('out2.csv', header = False, index = False, encoding = 'utf_8_sig')